In [ ]:
# Author: Karen Feng

In [13]:
import numpy as np
import pandas as pd

In [23]:
pt_meddra_col = 'pt_meddra_id'
# pt_meddra_col = 'pt_meddra_term'

In [43]:
onsides_adverse_reactions = pd.read_csv(
    '../data/onsides_20230309/adverse_reactions_active_labels.csv')[[
    'ingredients_rxcuis', pt_meddra_col]].astype('str')
onsides_adverse_reactions = onsides_adverse_reactions.groupby(
    'ingredients_rxcuis')[pt_meddra_col].apply(set).apply(';'.join).reset_index()
onsides_adverse_reactions = onsides_adverse_reactions.rename(columns={pt_meddra_col: 'adverse_reactions'})

In [44]:
onsides_boxed_warnings = pd.read_csv(
    '../data/onsides_20230309/boxed_warnings_active_labels.csv')[[
    'ingredients_rxcuis', pt_meddra_col]].astype('str')
onsides_boxed_warnings = onsides_boxed_warnings.groupby(
    'ingredients_rxcuis')[pt_meddra_col].apply(set).apply(';'.join).reset_index()
onsides_boxed_warnings = onsides_boxed_warnings.rename(columns={pt_meddra_col: 'boxed_warnings'})

In [45]:
input_df = pd.read_csv(
    '../data/conmed_example_data_with_best_rxcuid.csv',
    index_col=0, encoding='latin-1').fillna(np.nan).replace([np.nan], [None])

In [46]:
input_df['Best RxNorm Id'] = [
    str(int(rxcui)) if rxcui else None for rxcui in input_df['Best RxNorm Id']]

In [47]:
onsides = onsides_adverse_reactions.merge(
    onsides_boxed_warnings,
    how='outer',
    on='ingredients_rxcuis')

In [48]:
output_df = input_df.merge(
    onsides,
    how='left',
    left_on='Best RxNorm Id',
    right_on='ingredients_rxcuis')

In [49]:
onsides_adverse_reactions

,ingredients_rxcuis,adverse_reactions
0,1000082,10030041;10023644;10012434;10015946;10015993;1...
1,1005921,10000496;10037844;10000081;10028813;10019211;1...
2,"1006258, 1006297, 1006310, 1014385, 1014391, 1...",10011224;10016825;10024769;10010741;10002199;1...
3,"1006297, 1006506, 1006510, 1010965, 1010971, 1...",10041232;10011224;10015150;10024769;10002199;1...
4,"1006297, 214355, 314324, 314415, 314443, 31448...",10011224;10015150;10024769;10021097;10024855;1...
...,...,...
1928,9947,10043554;10040738;10011224;10016766;10036653;1...
1929,9949,10043554;10038236;10041660;10016825;10020642;1...
1930,996051,10021333;10034620;10037596;10048959;10040070;1...
1931,9997,10043554;10073508;10008570;10024264;10019211;1...


In [22]:
output_df.to_csv('../data/conmed_example_data_with_onsides_ids.csv')